In [4]:
import deepxde as dde
import numpy as np
from deepxde.backend import tf
import matplotlib.pyplot as plt

In [5]:
def ide(x, y, int_mat):
    rhs = tf.matmul(int_mat, y)
    lhs1 = tf.gradients(y, x)[0]
    return (lhs1 + y)[: tf.size(rhs)] - rhs

In [6]:
def kernel(x, s):
    return np.exp(s - x)

In [7]:
def func(x):
    return np.exp(-x) * np.cosh(x)

In [8]:
geom = dde.geometry.TimeDomain(0, 5)
ic = dde.icbc.IC(geom, func, lambda _, on_initial: on_initial)

In [13]:
quad_deg = 20
data = dde.data.IDE(
    geom,
    ide,
    ic,
    quad_deg,
    kernel=kernel,
    num_domain=10,
    num_boundary=2,
    train_distribution="uniform",
)

In [14]:
layer_size = [1] + [20] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

In [26]:
model = dde.Model(data, net)
model.compile("adam", lr=0.01)
model.train(iterations=50000)

Compiling model...
'compile' took 0.207895 s

Training model...

0         [6.85e-02, 1.00e+00]    [6.85e-02, 0.00e+00]    []  
1000      [1.04e-06, 7.19e-12]    [1.04e-06, 0.00e+00]    []  
2000      [6.14e-07, 9.23e-08]    [6.14e-07, 0.00e+00]    []  
3000      [6.16e-07, 6.57e-12]    [6.16e-07, 0.00e+00]    []  
4000      [5.31e-07, 3.55e-15]    [5.31e-07, 0.00e+00]    []  
5000      [1.19e-05, 1.32e-07]    [1.19e-05, 0.00e+00]    []  
6000      [5.05e-07, 6.87e-07]    [5.05e-07, 0.00e+00]    []  
7000      [2.46e-07, 1.13e-10]    [2.46e-07, 0.00e+00]    []  
8000      [1.16e-06, 2.78e-06]    [1.16e-06, 0.00e+00]    []  
9000      [2.00e-07, 1.13e-06]    [2.00e-07, 0.00e+00]    []  
10000     [1.03e-07, 4.72e-07]    [1.03e-07, 0.00e+00]    []  
11000     [4.76e-08, 5.68e-14]    [4.76e-08, 0.00e+00]    []  
12000     [5.08e-08, 2.40e-12]    [5.08e-08, 0.00e+00]    []  
13000     [4.01e-08, 5.12e-13]    [4.01e-08, 0.00e+00]    []  
14000     [4.58e-05, 1.26e-04]    [4.58e-05, 0.00e+00

(<deepxde.model.LossHistory at 0x179139d50>,
 <deepxde.model.TrainState at 0x1790b0f10>)

In [ ]:
X = geom.uniform_points(100)
y_true = func(X)
y_pred = model.predict(X)
print("L2 relative error:", dde.metrics.l2_relative_error(y_true, y_pred))

In [ ]:
plt.figure()
plt.plot(X, y_true, "-")
plt.plot(X, y_pred, "o")
plt.show()
np.savetxt("txt.dat", np.hstack((X, y_true, y_pred)))